In [1]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings("ignore")

# Read percentile reference data

In [2]:
age_group_score_into_percentile_df = pd.read_csv("data_for_IQ/age_group_into_percentile.csv")
age_group_score_into_percentile_df.head()

,rawScore,25,30,35,40,45,50,55,60,65
0,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0
1,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0
2,15,NaN,NaN,NaN,NaN,2.5,3.0,5.0,8.0,10.0
3,16,NaN,NaN,NaN,NaN,3.0,3.5,7.0,10.0,13.0
4,17,NaN,NaN,NaN,2.0,4.0,5.0,8.0,13.0,17.0


# Read IQ reference data

In [3]:
percentile_into_iq_df = pd.read_csv("data_for_IQ/percentile_into_iq.csv")
percentile_into_iq_df.set_index("percentile", inplace=True)
percentile_into_iq_df.head()

,IQ
percentile,
1.1,66
1.5,67
2.0,69
2.5,71
3.0,72


# read raw score data

ต้องมี score column

In [4]:
personal_data = pd.read_csv("data_for_IQ/example_data.csv")
personal_data.head()

,gender,age,score
0,ชาย,22,52
1,ชาย,22,53
2,ชาย,22,38
3,ชาย,22,51
4,หญิง,22,52


# วิธีการคิด percentile และ IQ โดยการปัดเศษลงทั้งหมด

In [5]:
age_groups = np.array([25, 30, 35, 40, 45, 50, 55, 60, 65, np.inf])

def calculate_percentile_by_looking_age_from_left_side(age, score):
    # Get age group.
    age_group = age_groups[age <= age_groups][0]
    if age_group == np.inf: # If age more than 65 than assume to be 65.
        age_group = 65
    age_group = str(int(age_group))

    # Get the target age group score into percentile dataframe.
    score_into_percentile_df = age_group_score_into_percentile_df.loc[
        :, ["rawScore", age_group]
    ].dropna().reset_index(drop=True)
    
    # Get the boundary of score for the age group.
    minimum_score = min(score_into_percentile_df["rawScore"])
    maximum_score = max(score_into_percentile_df["rawScore"])

    # If score less than minimum score return last percentile - 1 and if score more than maximum score return 99 percentile.
    if score < minimum_score:
        return max(score_into_percentile_df.loc[0, age_group] - 1, 0)
    if score > maximum_score:
        return 99
    
    # Return percentile from the table.
    return score_into_percentile_df.loc[
        score_into_percentile_df["rawScore"] == score, age_group
    ].values[0]

## compute percentile 

In [6]:
personal_data["percentileByLookAgeFromLeftSide"] = personal_data.apply(
    lambda row: int(calculate_percentile_by_looking_age_from_left_side(row["age"], row["score"])), axis=1
)
personal_data.head()

,gender,age,score,percentileByLookAgeFromLeftSide
0,ชาย,22,52,82
1,ชาย,22,53,85
2,ชาย,22,38,28
3,ชาย,22,51,78
4,หญิง,22,52,82


## compute IQ

In [7]:
personal_data["IQByLookAgeFromLeftSide"] = personal_data[
    "percentileByLookAgeFromLeftSide"
].map(percentile_into_iq_df["IQ"])
personal_data.head()

,gender,age,score,percentileByLookAgeFromLeftSide,IQByLookAgeFromLeftSide
0,ชาย,22,52,82,113
1,ชาย,22,53,85,115
2,ชาย,22,38,28,91
3,ชาย,22,51,78,111
4,หญิง,22,52,82,113


In [40]:
personal_data.head()

,gender,age,score,percentileByLookAgeFromLeftSide,IQByLookAgeFromLeftSide
0,ชาย,22,52,82,113
1,ชาย,22,53,85,115
2,ชาย,22,38,28,91
3,ชาย,22,51,78,111
4,หญิง,22,52,82,113


# วิธีการคิด IQ โดยการ Map the score to the IQ from the table by finding the closest age group

In [41]:
age_groups = np.array([25, 30, 35, 40, 45, 50, 55, 60, 65])

def calculate_percentile_by_closest_age_group(age, score):
    age_groups_diff = abs(age_groups - age)
    age_group_diff = age_groups[np.argmin(age_groups_diff)]
    
    # Get the target age group score into percentile dataframe.
    score_into_percentile_df = age_group_score_into_percentile_df.loc[
        :, ["rawScore", str(age_group_diff)]
    ].dropna().reset_index(drop=True)

    # Get the boundary of score for the age group.
    minimum_score = min(score_into_percentile_df["rawScore"])
    maximum_score = max(score_into_percentile_df["rawScore"])

    # If score less than minimum score return last percentile - 1 and if score more than maximum score return 99 percentile.
    if score < minimum_score:
        return max(score_into_percentile_df.loc[0, str(age_group_diff)] - 1, 0)
    if score > maximum_score:
        return 99
    
    # Return percentile from the table.
    return score_into_percentile_df.loc[
        score_into_percentile_df["rawScore"] == score, str(age_group_diff)
    ].values[0]

## คิด percentile ของแต่ละคน

In [43]:
personal_data["percentileByClosestAgeGroup"] = personal_data.apply(
    lambda row: int(
        calculate_percentile_by_closest_age_group(row["age"], row["score"])
    ),
    axis=1,
)

## คิด IQ ของแต่ละคนจาก percentile

In [44]:
personal_data["IQByClosestAgeGroup"] = personal_data["percentileByClosestAgeGroup"].map(
    percentile_into_iq_df["IQ"]
)

In [46]:
personal_data

,gender,age,score,percentileByLookAgeFromLeftSide,IQByLookAgeFromLeftSide,percentileByClosestAgeGroup,IQByClosestAgeGroup
0,ชาย,22,52,82,113,82,113
1,ชาย,22,53,85,115,85,115
2,ชาย,22,38,28,91,28,91
3,ชาย,22,51,78,111,78,111
4,หญิง,22,52,82,113,82,113
5,ชาย,21,54,88,118,88,118
6,ชาย,22,41,38,96,38,96
7,หญิง,22,47,61,104,61,104
8,หญิง,22,53,85,115,85,115
9,หญิง,22,52,82,113,82,113
